In [ ]:
import numpy as np
import pandas as pd
import gc

In [ ]:
filepath = 't4fbool.csv'

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
# 数组过滤
# tMat = np.mat(df[['density', 'ratio_suger', 'label']].values )
# tMat[tMat[:,2] == 0]

In [ ]:
ind = df.index[df['label'] == 0]
df0 = df.loc[ind]
labelMat0 = np.mat(df0[['label']].values)
del df0['label']
dataMat0 = np.mat(df0.values)

In [ ]:
del df0;
gc.collect()

In [ ]:

ind = df.index[df['label'] == 1]
df1 = df.loc[ind]

dataMat1 = np.mat(df1[['label']].values)
del df1['label']
labelMat1 = np.mat(df1.values)

In [ ]:
del df1;
gc.collect()

In [ ]:
"""
input:
np.cov(dataMat) : dataMat 每一列为一个样本，行为属性
    dataMat: 每一行为一个样本，列为属性
output:
    协方差矩阵,u均值向量
"""
def myCov(dataMat):
    r, c = dataMat.shape
    u = np.mean(dataMat, axis = 0)
    X = dataMat - u
    
    return (X.T * X) / (r - 1), u

In [ ]:
"""
np.linalg.svd()
input:
    dataMat:数据矩阵
output:
    U,V,sigma
"""
def mySvd(dataMat):
    lambu, U = np.linalg.eig(dataMat * dataMat.T)
    sortedind = np.argsort(lambu)
    ind = sortedind[::-1]
    U_sorted = U.T[ind].T
#     print(lambu[ind])
    lambv, V = np.linalg.eig(dataMat.T * dataMat)
    sortedind = np.argsort(lambv)
    ind = sortedind[::-1]
    V_sorted = V.T[ind].T
#     print(lambv[ind])
    lamb = np.power(lambu, 0.5)
#     print(lamb)
    return U_sorted, lamb, V_sorted

In [ ]:
"""
input : 
    U,V,sigma,d
output:
    appData:近似值
    appDataI : 近似值的逆
"""
def appDataI(U, sigma, V, d):
    sortedind = np.argsort(sigma)
    ind = sortedind[:-d-1:-1]
    sigma = sigma[ind]
    sigmaArr = np.zeros([d,d])
    for i in range(d):
        sigmaArr[i][i] = sigma[i]
    sigmaMat = np.mat(sigmaArr)
    appData = U[:, :d] * sigmaMat * V[:, :d].T
#     print('appData:', appData)
    appDataI = V[:, :d] * sigmaMat.I * U[:, :d].T
    return appDataI

In [ ]:
sigma0, u0 = myCov(dataMat0)
sigma1, u1 = myCov(dataMat1)
Sw = sigma0 + sigma1
# print(Sw)
U, sigma, V = mySvd(Sw)
SwI = appDataI(U, sigma, V, 10)
w = SwI * (u0 - u1).T

In [ ]:
dfw = pd.DataFrame(w)

In [ ]:
dfw.to_csv('classfyW.csv', index = False, encoding = 'utf-8')